# DMS Data Processing
### The Data
This notebooks serves as a step-by-step explanation of how the deep mutational scanning (DMS) data sets from Bloom lab at Fred Hutch and Starr lab at University of Utah were processed for our machine learning projects. Here, six data sets are utilized from 3 repos:
- [SARS-CoV-2-RBD_DMS_variants](https://github.com/jbloomlab/SARS-CoV-2-RBD_DMS_variants), paper: [Shifting mutational constraints in the SARS-CoV-2 receptor-binding domain during viral evolution](https://www.science.org/doi/10.1126/science.abo7896)
    - [Binding](https://github.com/jbloomlab/SARS-CoV-2-RBD_DMS_variants/blob/main/results/binding_Kd/bc_binding.csv)
    - [Expression](https://github.com/jbloomlab/SARS-CoV-2-RBD_DMS_variants/blob/main/results/expression_meanF/bc_expression.csv)
    - Variant backgrounds included:
        - Wuhan-Hu-1 background ("wildtype")
        - N501Y (Alpha, found in the B.1.1.7 lineage)
        - K417N+E484K+N501Y (Beta, found in the B.1.351 lineage)
        - E484K (Eta, found in the B.1.525 lineage)
- [SARS-CoV-2-RBD_Delta](https://github.com/jbloomlab/SARS-CoV-2-RBD_Delta/tree/main)
    - [Binding](https://github.com/jbloomlab/SARS-CoV-2-RBD_Delta/blob/main/results/binding_Kd/bc_binding.csv)
    - [Expression](https://github.com/jbloomlab/SARS-CoV-2-RBD_Delta/blob/main/results/expression_meanF/bc_expression.csv)
    - Variant backgrounds included:
        - L452R, T478K (Delta, found in the B.1.617.2 lineage)
- [SARS-CoV-2-RBD_DMS_Omicron](https://github.com/jbloomlab/SARS-CoV-2-RBD_DMS_Omicron), paper: [Deep mutational scans for ACE2 binding, RBD expression, and antibody escape in the SARS-CoV-2 Omicron BA.1 and BA.2 receptor-binding domains](https://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1010951)
    - [Binding](https://github.com/jbloomlab/SARS-CoV-2-RBD_DMS_Omicron/tree/main/results/binding_Kd)
    - [Expression](https://github.com/jbloomlab/SARS-CoV-2-RBD_DMS_Omicron/tree/main/results/expression_meanF)
    - Variant backgrounds included:
        - Wuhan-Hu-1
        - Omicron BA.1
        - Omicron BA.2
- [SARS-CoV-2-RBD_DMS_Omicron-XBB-BQ](https://github.com/tstarrlab/SARS-CoV-2-RBD_DMS_Omicron-XBB-BQ), paper: [Deep mutational scans of XBB.1.5 and BQ.1.1 reveal ongoing epistatic drift during SARS-CoV-2 evolution](https://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1011901)
    - [Binding](https://github.com/tstarrlab/SARS-CoV-2-RBD_DMS_Omicron-XBB-BQ/tree/main/results/binding_Kd)
    - [Expression](https://github.com/tstarrlab/SARS-CoV-2-RBD_DMS_Omicron-XBB-BQ/tree/main/results/expression_meanF)
    - Variant backgrounds included:
        - Omicron XBB.1.5
        - Omicron BQ.1.1

--- 

## Processing Steps

In [68]:
import os
import re
import pandas as pd
import numpy as np

Let's load in the data. We'll start with `variants-bc_binding.csv`.

In [69]:
data_dir = './data'

# Load in .csv
variants_binding_csv = os.path.join(data_dir, 'original_dms/variants-bc_binding.csv')
variants_binding_df = pd.read_csv(variants_binding_csv, sep=',', header=0)
variants_binding_df

,library,barcode,target,variant_class,aa_substitutions,n_aa_substitutions,TiteSeq_avgcount,log10Ka
0,pool1A,AAAAAAAAAAAAAGAA,N501Y,1 nonsynonymous,T46Q,1,1.955927,NaN
1,pool1A,AAAAAAAAAAAGGAGA,Wuhan_Hu_1,1 nonsynonymous,G166M,1,0.000000,NaN
2,pool1A,AAAAAAAAAAATTTAA,Wuhan_Hu_1,wildtype,NaN,0,36.004828,8.506648
3,pool1A,AAAAAAAAAACGCGTA,Wuhan_Hu_1,1 nonsynonymous,E154T,1,26.968699,8.586023
4,pool1A,AAAAAAAAAACTCCAA,Wuhan_Hu_1,1 nonsynonymous,F156M,1,40.906723,7.698023
...,...,...,...,...,...,...,...,...
520334,pool2A,TTTTTTTACCGCACTA,B1351,1 nonsynonymous,S19T,1,36.790470,8.275822
520335,pool2A,TTTTTTTAGCATTACC,E484K,wildtype,NaN,0,98.541470,8.312709
520336,pool2A,TTTTTTTGATATTGGA,Wuhan_Hu_1,1 nonsynonymous,C158G,1,38.976891,5.333362
520337,pool2A,TTTTTTTGGGCATGTA,N501Y,>1 nonsynonymous,P54D L183H,2,5.054723,NaN


Let's rename the target column for easier identification.

In [70]:
rename_map = {
    'N501Y': 'Alpha',
    'B1351': 'Beta',
    'Delta': 'Delta',
    'E484K': 'Eta',
    'BA1': 'Omicron_BA1',
    'BA2': 'Omicron_BA2',
    'BQ11': 'Omicron_BQ11',
    'XBB15': 'Omicron_XBB15',
    'Wuhan_Hu_1': 'Wuhan_Hu_1'
}

# Rename target columns for easier variant identification
variants_binding_df['target'] = variants_binding_df['target'].replace(rename_map)

Let's drop any NaN values in the column that contains the `log10Ka` values.

In [71]:
# Remove rows where specified column is NA/N
variants_binding_df = variants_binding_df.dropna(subset=['log10Ka']).reset_index(drop=True)
print(len(variants_binding_df))

431304


Let's take a look at how many of each `variant_class` there are. In the end, we want to filter out any variant classes that are not nonsynonymous mutations.

In [72]:
# Count number of entries per variant class
value_counts = variants_binding_df['variant_class'].value_counts()
print(value_counts)

# Filter out variant classes that are not nonsynonymous
nonsynonymous_df = variants_binding_df[variants_binding_df['variant_class'].str.contains('nonsynonymous', case=False, na=False)]
print(len(nonsynonymous_df))

variant_class
1 nonsynonymous     265843
wildtype            106281
>1 nonsynonymous     57840
synonymous            1340
Name: count, dtype: int64
323683


Now we want to acquire the unique nonsynonymous mutations. We create a column called "label" that consists of the "target" and "aa_substitutions" columns to prevent removal of the same substitutions that occurred for different targets. We'll then group by the "label" column and calculate the mean of the "log10Ka" for each grouping. We'll then merge with the duplicates replaced with the mean "log10Ka", and drop any duplicate label values. This leaves us with our unique nonsynonymous mutation data set.

In [73]:
nonsynonymous_df = nonsynonymous_df.copy()
print(f"Different variants/targets: {nonsynonymous_df['target'].unique()}\n")

# Add '_' to aa_substitutions column
nonsynonymous_df.loc[:, 'aa_substitutions'] = nonsynonymous_df['aa_substitutions'].replace(' ', '_', regex=True)

# Create a new column 'label' using 'target' and 'aa_substitutions' columns
nonsynonymous_df['label'] = nonsynonymous_df['target'] + '-' + nonsynonymous_df['aa_substitutions']
print(f"{nonsynonymous_df['label'].value_counts()}\n")

# Group by 'label' and calculate the mean of the specified value column for each group
unique_nonsynonymous_df = nonsynonymous_df.groupby('label', as_index=False)['log10Ka'].mean()
print(len(unique_nonsynonymous_df))

# Merge dfs, replace duplicate with average
merged_df = pd.merge(unique_nonsynonymous_df, nonsynonymous_df.drop(columns='log10Ka'), on='label', how='left')
print(len(merged_df))

# Drop duplicate rows based on 'label'
merged_df = merged_df.drop_duplicates(subset='label').reset_index(drop=True)
print(len(merged_df))

# Count number of unique nonsynonymous mutations
unique_nonsynonymous_mutations_counts = merged_df['label'].nunique()
print(f"Number of unique nonsynonymous mutations: {unique_nonsynonymous_mutations_counts}")

Different variants/targets: ['Wuhan_Hu_1' 'Beta' 'Eta' 'Alpha']

label
Eta-V115L                 153
Eta-N171S                 145
Beta-V32L                 135
Eta-V115S                 128
Eta-V115R                 127
                         ... 
Beta-L131P_D137G            1
Wuhan_Hu_1-L57M_N130K       1
Beta-G117K_Q168D_C195F      1
Eta-Y50R_Y123Q              1
Alpha-S19D_Y159H            1
Name: count, Length: 52763, dtype: int64

52763
323683
52763
Number of unique nonsynonymous mutations: 52763


We need to pull the different variant DNA sequences from the `/data` folders in each of the git repos... for example they look like `PacBio_amplicon_*.gb`, where * is the variant. We want to extract the DNA sequences, convert to RNA, then convert to amino acids. We then use each of these amino acid sequences as the reference sequence to apply the `aa_substitutions` to based on the variant, or `target`.

In [74]:
from Bio import SeqIO
from Bio.Seq import Seq

def extract_refseqs(variants: list) -> dict:
    rename_map = {
    'N501Y': 'Alpha',
    'B1351': 'Beta',
    'Delta': 'Delta',
    'E484K': 'Eta',
    'BA1': 'Omicron_BA1',
    'BA2': 'Omicron_BA2',
    'BQ11': 'Omicron_BQ11',
    'XBB15': 'Omicron_XBB15',
    'Wuhan_Hu_1': 'Wuhan_Hu_1'
    }
    target_dict = {}

    for v in variants:
        amplicon = os.path.join(data_dir, f'amplicons/PacBio_amplicon_{v}.gb')

        # Extract data from genbank file
        data = SeqIO.read(amplicon, 'genbank')

        # Get full sequence, features 
        full_seq = data.seq
        features = data.features

        # Get locations for the "gene", apply to the full sequence
        for feature in features:
            if feature.type == "gene":
                dna_seq = feature.location.extract(full_seq)

        # Convert DNA to RNA
        rna_seq = dna_seq.transcribe()

        # Translate RNA to Amino Acids
        protein_seq= rna_seq.translate()

        # Get the renamed target from the rename_map, default to the original name if not found
        renamed_target = rename_map.get(v, v)

        # Add to target dictionary with the renamed key
        target_dict[renamed_target] = protein_seq
    
    return target_dict

variants = ['Wuhan_Hu_1', 'B1351', 'E484K', 'N501Y']
variants_target_refseqs = extract_refseqs(variants)

for t in variants_target_refseqs: print(f"{t}: {variants_target_refseqs[t]}") 

Wuhan_Hu_1: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST
Beta: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKST
Eta: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST
Alpha: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKST


Now that we have the reference sequences for each of the 

In [75]:
def label_to_seq(row, target_refseqs:dict) -> str:
    """ Generate sequence based on reference sequence and aa_substitutions. """
    aa_substitutions, target = row['aa_substitutions'], row['target']
    refseq = list(target_refseqs[target])
    seq = refseq.copy()
    p = '([0-9]+)'
    
    if '_' in aa_substitutions:
        for mutcode in aa_substitutions.split('_'):
            [ori, pos, mut] = re.split(p, mutcode)
            pos = int(pos)-1    # use 0-based counting
            #assert refseq[pos].upper() == ori, f"At {pos}: {refseq[pos]} != {ori}"
            if refseq[pos].upper() != ori: print(f"{target}, At {pos}: {refseq[pos]} != {ori}")
            seq[pos] = mut.upper()
        seq = ''.join(seq)
        return seq

    if aa_substitutions=='': return ''.join(seq)

    [ori, pos, mut] = re.split(p, aa_substitutions)
    pos = int(pos)-1    # use 0-based counting
    #assert refseq[pos].upper() == ori, f"At {pos}: {refseq[pos]} != {ori}"
    if refseq[pos].upper() != ori: print(f"{target}, At {pos}: {refseq[pos]} != {ori}")
    seq[pos] = mut.upper()
    seq = ''.join(seq)    
    return seq

In [76]:
# Filter to only the columns we want and copy the DataFrame to avoid SettingWithCopyWarning
unique_filtered_df = merged_df[['label','target', 'aa_substitutions', 'log10Ka']].copy()

# Utilize 'aa_substitutions' to generate the mutated sequence
unique_filtered_df['sequence'] = unique_filtered_df.apply(label_to_seq, axis=1, target_refseqs=variants_target_refseqs)
unique_filtered_df

,label,target,aa_substitutions,log10Ka,sequence
0,Alpha-A105C,Alpha,A105C,9.086811,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
1,Alpha-A105C_G196Q,Alpha,A105C_G196Q,9.361016,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
2,Alpha-A105C_L162Q,Alpha,A105C_L162Q,6.275086,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
3,Alpha-A105D,Alpha,A105D,7.315937,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
4,Alpha-A105D_L111G,Alpha,A105D_L111G,7.150618,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
...,...,...,...,...,...
52758,Wuhan_Hu_1-Y93V_A145V,Wuhan_Hu_1,Y93V_A145V,5.000000,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
52759,Wuhan_Hu_1-Y93V_K128L,Wuhan_Hu_1,Y93V_K128L,5.827319,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
52760,Wuhan_Hu_1-Y93V_N110I,Wuhan_Hu_1,Y93V_N110I,5.263860,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...
52761,Wuhan_Hu_1-Y93W,Wuhan_Hu_1,Y93W,5.293445,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...


In [77]:
# Final file pathing
save_as = os.path.join(data_dir, 'processed_dms/mutation_variants-bc_binding.csv')

# Select final columns 
final_df = unique_filtered_df[['label', 'target', 'log10Ka', 'sequence']].copy()

# Save processed data to .csv
final_df.to_csv(save_as, index=False)

Now, let's run this process but on each of the other data sets:

In [78]:
def process_data(target_refseqs, input_csv, output_csv):
    """ 
    Process the DMS binding and expression datasets.
    - Drop NAs
    - Select only nonsynonymous variant classes
    - Select unique nonsynonymous mutations
        - For duplicate aa_substitutions, take the mean value of log10Ka or expression (expression?)
    - Apply mutations to reference sequence
    """

    if  "binding" in input_csv:
        value_column = "log10Ka"
        print("Looking at DMS binding dataset.")
    elif "expression" in input_csv:
        value_column = "expression"
        print("Looking at DMS expression dataset.")

    full_df = pd.read_csv(input_csv, sep=',', header=0)
    row_count = len(full_df)
    print(f"Number of data points: {row_count}")

    rename_map = {
    'N501Y': 'Alpha',
    'B1351': 'Beta',
    'Delta': 'Delta',
    'E484K': 'Eta',
    'BA1': 'Omicron_BA1',
    'BA2': 'Omicron_BA2',
    'BQ11': 'Omicron_BQ11',
    'XBB15': 'Omicron_XBB15',
    'Wuhan_Hu_1': 'Wuhan_Hu_1'
    }
    # Rename target columns for easier variant identification
    full_df['target'] = full_df['target'].replace(rename_map)

    # Remove rows where specified column is NA
    full_df = full_df.dropna(subset=[value_column]).reset_index(drop=True)
    print(f"Number of data points with na: {row_count-len(full_df)}")
    print(f"Number of data points left {len(full_df)}")
    
    # Count number of entries per variant class
    value_counts = full_df["variant_class"].value_counts()
    print(f"{value_counts}\n")
    # Filter out variant classes that are not nonsynonymous
    nonsynonymous_df = full_df[full_df['variant_class'].str.contains('nonsynonymous', case=False, na=False)]
    # Copy the DataFrame to avoid SettingWithCopyWarning
    nonsynonymous_df = nonsynonymous_df.copy()

    print(f"Different variants/targets: {nonsynonymous_df['target'].unique()}\n")

    # Add '_' to aa_substitutions column
    nonsynonymous_df.loc[:, 'aa_substitutions'] = nonsynonymous_df['aa_substitutions'].replace(' ', '_', regex=True)
    # Create a new column 'label' using 'target' and 'aa_substitutions' columns
    nonsynonymous_df['label'] = nonsynonymous_df['target'] + '-' + nonsynonymous_df['aa_substitutions']
    print(f"{nonsynonymous_df['label'].value_counts()}\n")
    # Group by 'label' and calculate the mean of the specified value column for each group
    unique_nonsynonymous_df = nonsynonymous_df.groupby('label', as_index=False)[value_column].mean()
    # Merge dfs, replace duplicate with average
    merged_df = pd.merge(unique_nonsynonymous_df, nonsynonymous_df.drop(columns=value_column), on='label', how='left')
    # Drop duplicate rows based on 'label'
    merged_df = merged_df.drop_duplicates(subset='label').reset_index(drop=True)
    # Count number of unique nonsynonymous mutations
    unique_nonsynonymous_mutations_counts = merged_df['label'].nunique()
    print(f"Number of unique nonsynonymous mutations: {unique_nonsynonymous_mutations_counts}")
    
    # Filter to only the columns we want and copy the DataFrame to avoid SettingWithCopyWarning
    unique_filtered_df = merged_df[['label','target', 'aa_substitutions', value_column]].copy()
    # Utilize 'aa_substitutions' to generate the mutated sequence
    unique_filtered_df['sequence'] = unique_filtered_df.apply(label_to_seq, axis=1, target_refseqs=target_refseqs)

    # Select final columns 
    final_df = unique_filtered_df[['label', 'target', value_column, 'sequence']].copy()
    # Save processed data to .csv
    final_df.to_csv(output_csv, index=False)

In [79]:
variants = ['Wuhan_Hu_1', 'B1351', 'E484K', 'N501Y', 'Delta', 'BA1', 'BA2', 'BQ11', 'XBB15']
target_refseqs = extract_refseqs(variants)
for t in target_refseqs: print(f"{t}: {target_refseqs[t]}") 

Wuhan_Hu_1: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST
Beta: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKST
Eta: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVKGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKST
Alpha: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKST
Delta: NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYRYRLFRKSNLKPFERDISTEIYQAGSKPCNGVEG

In [80]:
expression_csv = os.path.join(data_dir, 'original_dms/variants-bc_expression.csv')
save_as = os.path.join(data_dir, 'processed_dms/mutation_variants-bc_expression.csv')
process_data(target_refseqs, expression_csv, save_as)

Looking at DMS expression dataset.
Number of data points: 337629
Number of data points with na: 43311
Number of data points left 294318
variant_class
1 nonsynonymous     178356
wildtype             73795
>1 nonsynonymous     40420
synonymous             969
stop                   778
Name: count, dtype: int64

Different variants/targets: ['Wuhan_Hu_1' 'Beta' 'Eta' 'Alpha']

label
Eta-V115L                90
Beta-V32L                88
Eta-N171S                86
Eta-V115S                70
Eta-N171R                70
                         ..
Wuhan_Hu_1-G83M_S184K     1
Wuhan_Hu_1-E10D_E76V      1
Alpha-S43A_F160E          1
Beta-D34R_F126W           1
Alpha-P54D_L183H          1
Name: count, Length: 54391, dtype: int64

Number of unique nonsynonymous mutations: 54391


### Delta Dataset

In [81]:
binding_csv = os.path.join(data_dir, 'original_dms/delta-bc_binding.csv')
save_as = os.path.join(data_dir, 'processed_dms/mutation_delta-bc_binding.csv')
process_data(target_refseqs, binding_csv, save_as)

Looking at DMS binding dataset.
Number of data points: 205734
Number of data points with na: 23388
Number of data points left 182346
variant_class
1 nonsynonymous     120999
wildtype             36189
>1 nonsynonymous     24853
synonymous             305
Name: count, dtype: int64

Different variants/targets: ['Delta']

label
Delta-N64W           70
Delta-S45P           66
Delta-K114W          62
Delta-N110S          62
Delta-E154Y          61
                     ..
Delta-T63K_K94M       1
Delta-F126M_S139T     1
Delta-Q79M_Q168H      1
Delta-Y21C_K198C      1
Delta-Y50R_R127P      1
Name: count, Length: 27274, dtype: int64

Number of unique nonsynonymous mutations: 27274


In [82]:
expression_csv = os.path.join(data_dir, 'original_dms/delta-bc_expression.csv')
save_as = os.path.join(data_dir, 'processed_dms/mutation_delta-bc_expression.csv')
process_data(target_refseqs, expression_csv, save_as)

Looking at DMS expression dataset.
Number of data points: 205734
Number of data points with na: 11573
Number of data points left 194161
variant_class
1 nonsynonymous     127540
wildtype             37693
>1 nonsynonymous     26532
stop                  2076
synonymous             320
Name: count, dtype: int64

Different variants/targets: ['Delta']

label
Delta-N64W               71
Delta-S45P               69
Delta-F70R               67
Delta-K114W              64
Delta-R136L              63
                         ..
Delta-A42S_N64W           1
Delta-Y91D_Y165G          1
Delta-K26Q_C49H_C158D     1
Delta-A22K_S108F          1
Delta-G86M_G146Y          1
Name: count, Length: 28729, dtype: int64

Number of unique nonsynonymous mutations: 28729


### 1st Omicron Dataset

In [83]:
binding_csv = os.path.join(data_dir, 'original_dms/omicron_BA-bc_binding.csv')
save_as = os.path.join(data_dir, 'processed_dms/mutation_omicron_BA-bc_binding.csv')
process_data(target_refseqs, binding_csv, save_as)

Looking at DMS binding dataset.
Number of data points: 649481
Number of data points with na: 308459
Number of data points left 341022
variant_class
1 nonsynonymous     320786
>1 nonsynonymous     10613
wildtype              9269
synonymous             354
Name: count, dtype: int64

Different variants/targets: ['Omicron_XBB15' 'Omicron_BQ11' 'Omicron_BA2']

label
Omicron_XBB15-Y121S          103
Omicron_XBB15-Y121R           87
Omicron_XBB15-V180-           82
Omicron_BQ11-P149S            78
Omicron_XBB15-Y121T           77
                            ... 
Omicron_BA2-R179K_T201Q        1
Omicron_BQ11-A14T_I80M         1
Omicron_XBB15-R73F_A81S        1
Omicron_BA2-K48S_V52F          1
Omicron_XBB15-Y119F_R124T      1
Name: count, Length: 21653, dtype: int64

Number of unique nonsynonymous mutations: 21653


In [84]:
expression_csv = os.path.join(data_dir, 'original_dms/omicron_BA-bc_expression.csv')
save_as = os.path.join(data_dir, 'processed_dms/mutation_omicron_BA-bc_expression.csv')
process_data(target_refseqs, expression_csv, save_as)

Looking at DMS expression dataset.
Number of data points: 598394
Number of data points with na: 97861
Number of data points left 500533
variant_class
1 nonsynonymous     446359
wildtype             25152
>1 nonsynonymous     23048
stop                  5659
synonymous             315
Name: count, dtype: int64

Different variants/targets: ['Wuhan_Hu_1' 'Omicron_BA1' 'Omicron_BA2']

label
Omicron_BA1-S19W           119
Omicron_BA1-Y50G           116
Omicron_BA1-A18W           112
Omicron_BA1-A18Y           112
Omicron_BA1-S19G           111
                          ... 
Omicron_BA1-R27M_D137A       1
Wuhan_Hu_1-S45R_V52P         1
Wuhan_Hu_1-V11Q_S200Q        1
Omicron_BA2-E135N_G172S      1
Omicron_BA2-D68W_C158Y       1
Name: count, Length: 32663, dtype: int64

Number of unique nonsynonymous mutations: 32663


### 2nd Omicron Dataset

In [85]:
binding_csv = os.path.join(data_dir, 'original_dms/omicron_XBB_BQ-bc_binding.csv')
save_as = os.path.join(data_dir, 'processed_dms/mutation_omicron_XBB_BQ-bc_binding.csv')
process_data(target_refseqs, binding_csv, save_as)

Looking at DMS binding dataset.
Number of data points: 598394
Number of data points with na: 163488
Number of data points left 434906
variant_class
1 nonsynonymous     386015
wildtype             27387
>1 nonsynonymous     21159
synonymous             345
Name: count, dtype: int64

Different variants/targets: ['Wuhan_Hu_1' 'Omicron_BA1' 'Omicron_BA2']

label
Omicron_BA1-S19W          99
Omicron_BA1-A18Y          98
Omicron_BA1-A18W          97
Omicron_BA1-A18E          94
Omicron_BA1-Y66W          93
                          ..
Wuhan_Hu_1-F12G_E154H      1
Omicron_BA2-G51A_L111F     1
Omicron_BA1-D59Y_D68K      1
Omicron_BA1-A89S_A154Y     1
Omicron_BA2-D68W_C158Y     1
Name: count, Length: 31244, dtype: int64

Number of unique nonsynonymous mutations: 31244


In [86]:
expression_csv = os.path.join(data_dir, 'original_dms/omicron_XBB_BQ-bc_expression.csv')
save_as = os.path.join(data_dir, 'processed_dms/mutation_omicron_XBB_BQ-bc_expression.csv')
process_data(target_refseqs, expression_csv, save_as)

Looking at DMS expression dataset.
Number of data points: 649481
Number of data points with na: 244004
Number of data points left 405477
variant_class
1 nonsynonymous     375709
>1 nonsynonymous     13002
wildtype             11322
stop                  4990
synonymous             454
Name: count, dtype: int64

Different variants/targets: ['Omicron_XBB15' 'Omicron_BQ11' 'Omicron_BA2']

label
Omicron_XBB15-V180-          116
Omicron_XBB15-Y121S          112
Omicron_XBB15-Y121R           91
Omicron_BQ11-P149S            84
Omicron_XBB15-Y121T           75
                            ... 
Omicron_XBB15-K132-_E141G      1
Omicron_BQ11-F70L_S139C        1
Omicron_BA2-A22K_W23L          1
Omicron_XBB15-K94E_I138V       1
Omicron_XBB15-Y119F_R124T      1
Name: count, Length: 23680, dtype: int64

Number of unique nonsynonymous mutations: 23680


---

## Combination of Variant Datasets

Let's combine all expression and all binding data into 2 big files so we can put them in the ESM-BLSTM model. Get rid of all Wuhan variants. We'll add a column that represents the variant.

### Binding

In [87]:
delta_binding_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_delta-bc_binding.csv"), sep=',', header=0)
print(f"Delta binding # entries: {len(delta_binding_df)}")
delta_binding_df = delta_binding_df[delta_binding_df['target'] != 'Wuhan_Hu_1']
print(f"Delta binding # entries, no wuhan: {len(delta_binding_df)}")
print(f"Number of duplicates: {len(delta_binding_df[delta_binding_df.duplicated(subset=['label'], keep=False)])}\n")

omicron1_binding_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_omicron_BA-bc_binding.csv"), sep=',', header=0)
print(f"Omicron1 binding # entries: {len(omicron1_binding_df)}")
omicron1_binding_df = omicron1_binding_df[omicron1_binding_df['target'] != 'Wuhan_Hu_1']
print(f"Omicron1 binding # entries, no wuhan: {len(omicron1_binding_df)}")
print(f"Number of duplicates: {len(omicron1_binding_df[omicron1_binding_df.duplicated(subset=['label'], keep=False)])}\n")

omicron2_binding_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_omicron_XBB_BQ-bc_binding.csv"), sep=',', header=0)
print(f"Omicron2 binding # entries: {len(omicron2_binding_df)}")
omicron2_binding_df = omicron2_binding_df[omicron2_binding_df['target'] != 'Wuhan_Hu_1']
print(f"Omicron2 binding # entries, no wuhan: {len(omicron2_binding_df)}")
print(f"Number of duplicates: {len(omicron2_binding_df[omicron2_binding_df.duplicated(subset=['label'], keep=False)])}\n")

alpha_beta_eta_binding_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_variants-bc_binding.csv"), sep=',', header=0)
print(f"Alpha, beta, eta # entries: {len(alpha_beta_eta_binding_df)}")
alpha_beta_eta_binding_df = alpha_beta_eta_binding_df[alpha_beta_eta_binding_df['target'] != 'Wuhan_Hu_1']
print(f"Alpha, beta, eta # entries, no wuhan: {len(alpha_beta_eta_binding_df)}")
print(f"Number of duplicates: {len(alpha_beta_eta_binding_df[alpha_beta_eta_binding_df.duplicated(subset=['label'], keep=False)])}\n")

Delta binding # entries: 27274
Delta binding # entries, no wuhan: 27274
Number of duplicates: 0

Omicron1 binding # entries: 21653
Omicron1 binding # entries, no wuhan: 21653
Number of duplicates: 0

Omicron2 binding # entries: 31244
Omicron2 binding # entries, no wuhan: 17352
Number of duplicates: 0

Alpha, beta, eta # entries: 52763
Alpha, beta, eta # entries, no wuhan: 39314
Number of duplicates: 0



In [88]:
# Combination
combined_binding_df = pd.concat([delta_binding_df, omicron2_binding_df, omicron1_binding_df, alpha_beta_eta_binding_df], ignore_index=True)
combined_binding_df = combined_binding_df[['label', 'log10Ka', 'sequence']]
print(f"Total number of combined entries: {len(combined_binding_df)}")

# Identify all duplicate entries based on certain columns, including the original entries
duplicated_df = combined_binding_df[combined_binding_df.duplicated(subset=['label', 'sequence'], keep=False)]
print(f"Number of entries with duplicates (all copies): {len(duplicated_df)}")

# Filter out these duplicate entries from the combined df
filtered_df = combined_binding_df[~combined_binding_df['label'].isin(duplicated_df['label'])].reset_index(drop=True)
print(f"Number of entries with no duplicates: {len(filtered_df)}")

# Group by 'label' and 'sequence', and calculate the mean of the specified value column for each group
deduplicated_df = duplicated_df.groupby(['label', 'sequence'], as_index=False)["log10Ka"].mean()
print(f"Number of deduplicated entries: {len(deduplicated_df)}")

# Combine the deduplicated df and filtered df
combined_binding_df = pd.concat([deduplicated_df, filtered_df], ignore_index=True)
print(f"New number of entries with no duplicates: {len(combined_binding_df)}")

# Check to see if duplicates were removed:
print(f"Confirm successful deduplication?: {len(combined_binding_df[combined_binding_df.duplicated(subset=['label', 'sequence'], keep=False)])}")

# Save
combined_binding_df.to_csv(os.path.join(data_dir, 'processed_dms/mutation_combined_binding.csv'), index=False)
combined_binding_df

Total number of combined entries: 105593
Number of entries with duplicates (all copies): 14192
Number of entries with no duplicates: 91401
Number of deduplicated entries: 7096
New number of entries with no duplicates: 98497
Confirm successful deduplication?: 0


,label,sequence,log10Ka
0,Omicron_BA2-A105C,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,8.783580
1,Omicron_BA2-A105D,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,6.315664
2,Omicron_BA2-A105D_G196S,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,6.727197
3,Omicron_BA2-A105D_N147M,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,5.653277
4,Omicron_BA2-A105D_S108I,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,6.083346
...,...,...,...
98492,Eta-Y93T,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,5.892614
98493,Eta-Y93V,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,5.776665
98494,Eta-Y93V_L122Y,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,5.000000
98495,Eta-Y93W,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,5.639833


### Expression

In [89]:
delta_expression_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_delta-bc_expression.csv"), sep=',', header=0)
print(f"Delta expression # entries: {len(delta_expression_df)}")
delta_expression_df = delta_expression_df[delta_expression_df['target'] != 'Wuhan_Hu_1']
print(f"Delta expression # entries, no wuhan: {len(delta_expression_df)}")
print(f"Number of duplicates: {len(delta_expression_df[delta_expression_df.duplicated(subset=['label'], keep=False)])}\n")

omicron1_expression_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_omicron_BA-bc_expression.csv"), sep=',', header=0)
print(f"Omicron1 expression # entries: {len(omicron1_expression_df)}")
omicron1_expression_df = omicron1_expression_df[omicron1_expression_df['target'] != 'Wuhan_Hu_1']
print(f"Omicron1 expression # entries, no wuhan: {len(omicron1_expression_df)}")
print(f"Number of duplicates: {len(omicron1_expression_df[omicron1_expression_df.duplicated(subset=['label'], keep=False)])}\n")

omicron2_expression_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_omicron_XBB_BQ-bc_expression.csv"), sep=',', header=0)
print(f"Omicron2 expression # entries: {len(omicron2_expression_df)}")
omicron2_expression_df = omicron2_expression_df[omicron2_expression_df['target'] != 'Wuhan_Hu_1']
print(f"Omicron2 expression # entries, no wuhan: {len(omicron2_expression_df)}")
print(f"Number of duplicates: {len(omicron2_expression_df[omicron2_expression_df.duplicated(subset=['label'], keep=False)])}\n")

alpha_beta_eta_expression_df = pd.read_csv(os.path.join(data_dir, "processed_dms/mutation_variants-bc_expression.csv"), sep=',', header=0)
print(f"Alpha, beta, eta # entries: {len(alpha_beta_eta_expression_df)}")
alpha_beta_eta_expression_df = alpha_beta_eta_expression_df[alpha_beta_eta_expression_df['target'] != 'Wuhan_Hu_1']
print(f"Alpha, beta, eta # entries, no wuhan: {len(alpha_beta_eta_expression_df)}")
print(f"Number of duplicates: {len(alpha_beta_eta_expression_df[alpha_beta_eta_expression_df.duplicated(subset=['label'], keep=False)])}\n")

Delta expression # entries: 28729
Delta expression # entries, no wuhan: 28729
Number of duplicates: 0

Omicron1 expression # entries: 32663
Omicron1 expression # entries, no wuhan: 19032
Number of duplicates: 0

Omicron2 expression # entries: 23680
Omicron2 expression # entries, no wuhan: 23680
Number of duplicates: 0

Alpha, beta, eta # entries: 54391
Alpha, beta, eta # entries, no wuhan: 40780
Number of duplicates: 0



In [90]:
# Combination
combined_expression_df = pd.concat([delta_expression_df, omicron2_expression_df, omicron1_expression_df, alpha_beta_eta_expression_df], ignore_index=True)
combined_expression_df = combined_expression_df[['label', 'expression', 'sequence']]
print(f"Total number of combined entries: {len(combined_expression_df)}")

# Identify all duplicate entries based on certain columns, including the original entries
duplicated_df = combined_expression_df[combined_expression_df.duplicated(subset=['label', 'sequence'], keep=False)]
print(f"Number of entries with duplicates (all copies): {len(duplicated_df)}")

# Filter out these duplicate entries from the combined df
filtered_df = combined_expression_df[~combined_expression_df['label'].isin(duplicated_df['label'])].reset_index(drop=True)
print(f"Number of entries with no duplicates: {len(filtered_df)}")

# Group by 'label' and 'sequence', and calculate the mean of the specified value column for each group
deduplicated_df = duplicated_df.groupby(['label', 'sequence'], as_index=False)["expression"].mean()
print(f"Number of deduplicated entries: {len(deduplicated_df)}")

# Combine the deduplicated df and filtered df
combined_expression_df = pd.concat([deduplicated_df, filtered_df], ignore_index=True)
print(f"New number of entries with no duplicates: {len(combined_expression_df)}")

# Check to see if duplicates were removed:
print(f"Confirm successful deduplication?: {len(combined_expression_df[combined_expression_df.duplicated(subset=['label', 'sequence'], keep=False)])}")

# Save
combined_expression_df.to_csv(os.path.join(data_dir, 'processed_dms/mutation_combined_expression.csv'), index=False)
combined_expression_df

Total number of combined entries: 112221
Number of entries with duplicates (all copies): 16240
Number of entries with no duplicates: 95981
Number of deduplicated entries: 8120
New number of entries with no duplicates: 104101
Confirm successful deduplication?: 0


,label,sequence,expression
0,Omicron_BA2-A105C,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,7.975114
1,Omicron_BA2-A105D,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,7.356103
2,Omicron_BA2-A105D_G196S,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,6.784158
3,Omicron_BA2-A105D_K114D,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,7.497698
4,Omicron_BA2-A105D_K114W,NITNLCPFDEVFNATRFASVYAWNRKRISNCVADYSVLYNFAPFFA...,7.589814
...,...,...,...
104096,Eta-Y93T,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,7.287253
104097,Eta-Y93V,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,7.620438
104098,Eta-Y93V_L122Y,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,7.187713
104099,Eta-Y93W,NITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFST...,7.246666


### Splitting the data for training/testing

In [91]:
import csv
import random

def split_csv(rnd_seed: int, input_csv: str, output_csv:str):
    """
    Split csv file into train, test data.
    """
    train_csv = output_csv.replace('.csv', '_train.csv')
    test_csv  = output_csv.replace('.csv', '_test.csv')

    with open(input_csv, "r") as input_file:
        reader = csv.reader(input_file)
        header = next(reader)
        input_records = list(reader)

    random.seed(rnd_seed)
    random.shuffle(input_records)

    split_idx = int(0.8 * len(input_records))
    train_records = input_records[:split_idx]
    test_records = input_records[split_idx:]

    with open(train_csv, 'w') as ft, open(test_csv, 'w') as fv:
        train_writer = csv.writer(ft)
        test_writer = csv.writer(fv)

        train_writer.writerow(header)
        test_writer.writerow(header)

        for record in train_records:
            train_writer.writerow(record)

        for record in test_records:
            test_writer.writerow(record)

    print(f'Total: {len(input_records)}, Train: {len(train_records)}, Test: {len(test_records)}')

In [92]:
# Split to train, test data (80/20)
rnd_seed = 0

combined_binding_csv = 'mutation_combined_binding.csv'
split_csv(
    rnd_seed, 
    os.path.join(data_dir, f'processed_dms/{combined_binding_csv}'), 
    os.path.join(data_dir, f'split_processed_dms/{combined_binding_csv}')
)

combined_expression_csv = 'mutation_combined_expression.csv'
split_csv(
    rnd_seed, 
    os.path.join(data_dir, f'processed_dms/{combined_expression_csv}'), 
    os.path.join(data_dir, f'split_processed_dms/{combined_expression_csv}')
)

Total: 98497, Train: 78797, Test: 19700
Total: 104101, Train: 83280, Test: 20821
